<a href="https://colab.research.google.com/github/thedoctorJJ/gitbub-demo/blob/master/Single_Audit_Auditor_FAC_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install requests beautifulsoup4 pandas openpyxl

from google.colab import drive
drive.mount('/content/drive')

import os
output_dir = "/content/drive/My Drive/FAC_Scraper"
os.makedirs(output_dir, exist_ok=True)

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

class FACSearchScraper:
    def __init__(self, output_dir="/content/drive/My Drive/FAC_Scraper"):
        self.base_url = "https://app.fac.gov"
        self.search_url = "https://app.fac.gov/dissemination/search/"
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        }
        self.output_dir = output_dir

    def get_csrf_token(self):
        """Get CSRF token from the search page"""
        response = self.session.get(self.search_url, headers=self.headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.find('input', {'name': 'csrfmiddlewaretoken'})['value']

    def get_audit_details(self, url):
        """Extract auditor details from an audit summary page"""
        print(f"Fetching details from: {url}")
        try:
            full_url = self.base_url + url if not url.startswith('http') else url
            response = self.session.get(full_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the auditor information section
            auditor_section = soup.find(lambda tag: tag.name == 'h2' and 'Auditor' in tag.text)
            if not auditor_section:
                return None

            # Get the table following the Auditor heading
            table = auditor_section.find_next('table')
            if not table:
                return None

            # Extract data from the table
            data = {}
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                for cell in cells:
                    # Look for specific fields
                    if strong := cell.find('strong'):
                        field = strong.text.strip().replace(':', '').strip()
                        value = cell.text.replace(strong.text, '').strip()
                        if field == 'Contact Name':
                            data['Auditor_Name'] = value
                        elif field == 'Contact title':
                            data['Title'] = value
                        elif field == 'Email':
                            data['Email'] = value
                        elif field == 'Phone':
                            data['Phone'] = value
                        elif field == 'City and state':
                            city_state = value.split(',')
                            if len(city_state) == 2:
                                data['City'] = city_state[0].strip()
                                data['State'] = city_state[1].strip()

            # Get auditee name from the page title
            auditee_name = soup.find('p', class_='margin-0 text-bold')
            if auditee_name:
                data['Auditee_Name'] = auditee_name.text.strip()

            return data

        except Exception as e:
            print(f"Error getting audit details: {e}")
            return None

    def search_audits(self, state='NY', audit_years=['2023', '2024'], limit=130):
        """Perform the search and get results with pagination"""
        csrf_token = self.get_csrf_token()
        print(f"Got CSRF token, submitting search...")

        results = []
        page = 1

        while len(results) < limit:
            print(f"\nProcessing page {page}...")

            form_data = {
                'csrfmiddlewaretoken': csrf_token,
                'audit_year': audit_years,
                'auditee_state': state,
                'page': str(page)
            }

            response = self.session.post(
                self.search_url,
                data=form_data,
                headers={**self.headers, 'Referer': self.search_url}
            )

            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all summary links on this page
            summary_links = []
            for link in soup.find_all('a', href=True):
                if '/dissemination/summary/' in link['href']:
                    summary_links.append(link['href'])

            if not summary_links:
                print("No more results found on this page")
                break

            print(f"Found {len(summary_links)} audit reports on this page")

            # Process each summary page
            for i, link in enumerate(summary_links, 1):
                if len(results) >= limit:
                    break

                print(f"\nProcessing audit {len(results) + 1} of {limit}")
                auditor_info = self.get_audit_details(link)
                if auditor_info:
                    auditor_info['Report_ID'] = link.split('/')[-1]
                    results.append(auditor_info)
                    print(f"Found auditor: {auditor_info.get('Auditor_Name', 'Unknown')}")
                time.sleep(1)  # Be nice to the server

            page += 1

        print(f"\nCompleted processing {len(results)} audit reports")
        return results

    # Update just the save_to_excel method in Cell 2
    def save_to_excel(self, results, state='NY'):
        """Save results to Excel with numbered index"""
        if results:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = os.path.join(self.output_dir, f'auditor_contacts_{state}_{timestamp}.xlsx')

            df = pd.DataFrame(results)

            # Add numbered index starting from 1
            df.insert(0, 'Number', range(1, len(df) + 1))

            # Reorder columns with Number first
            columns = ['Number', 'Report_ID', 'Auditee_Name', 'Auditor_Name', 'Title',
                      'Email', 'Phone', 'City', 'State']
            df = df.reindex(columns=columns)

            df.to_excel(filename, index=False)
            print(f"\nResults saved to {filename}")
            return df
        else:
            print("\nNo results to save")
            return None

# Run the scraper
scraper = FACSearchScraper()
results = scraper.search_audits(state='NY', audit_years=['2023', '2024'], limit=130)
df = scraper.save_to_excel(results)

if df is not None:
    print("\nFirst few results:")
    display(df.head())
    print(f"\nTotal audits found: {len(df)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Got CSRF token, submitting search...

Processing page 1...
Found 30 audit reports on this page

Processing audit 1 of 130
Fetching details from: /dissemination/summary/2023-06-GSAFAC-0000061985
Found auditor: John Eusanio

Processing audit 2 of 130
Fetching details from: /dissemination/summary/2023-12-GSAFAC-0000056856
Found auditor: Nicole Overbaugh

Processing audit 3 of 130
Fetching details from: /dissemination/summary/2023-12-GSAFAC-0000064457
Found auditor: Luis Carlos Rivera

Processing audit 4 of 130
Fetching details from: /dissemination/summary/2024-03-GSAFAC-0000056347
Found auditor: Cole Monroe

Processing audit 5 of 130
Fetching details from: /dissemination/summary/2024-04-GSAFAC-0000064230
Found auditor: TAMI RADINSKY

Processing audit 6 of 130
Fetching details from: /dissemination/summary/2024-06-GSAFAC-0000057315
Found auditor: ROBERT DANIELE

P

,Number,Report_ID,Auditee_Name,Auditor_Name,Title,Email,Phone,City,State
0,1,2023-06-GSAFAC-0000061985,"Community Development Project, Inc.",John Eusanio,Partner,jeusanio@citrincooperman.com,2126971000,New York,NY
1,2,2023-12-GSAFAC-0000056856,"National Young Farmers Coalition, Inc.",Nicole Overbaugh,Principal,noverbaugh@uhy-us.com,5187199349,Hudson,NY
2,3,2023-12-GSAFAC-0000064457,"AFTER HOURS PROJECT, INC.",Luis Carlos Rivera,Partner,luis@cpatrust.com,8456383113,New City,NY
3,4,2024-03-GSAFAC-0000056347,Village of North Syracuse Housing Authority,Cole Monroe,Principal,cmonroe@smco.cpa,6155453153,Franklin,TN
4,5,2024-04-GSAFAC-0000064230,"AMERICAN JEWISH WORLD SERVICE, INC",TAMI RADINSKY,AUDIT PARTNER,TAMI.RADINSKY@US.GT.COM,6468258431,NEW YORK,NY



Total audits found: 130
